In [1]:
import pandas as pd
from enceladus.workflows import TestingPipeline

worker = TestingPipeline(
    data_dir='/home/cam/Documents/database_tools/data/mimic3-data-20230205/data/',
    model_dir='heartfelt/Enceladus-notebooks/model-breezy-wind-80:v11',
    scaler_dir='/home/cam/Documents/database_tools/data/mimic3-data-20230205/data/min_max_scaler_1676826861609360307.pkl',
)

# returns rescaled data
error, ppg, vpg, apg, abp, pred = worker.run()

2023-02-20 13:27:36.762898: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-20 13:27:36.762909: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: camhpj (heartfelt). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-breezy-wind-80:v11, 58.89MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.1


2023-02-20 13:27:46.446803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-20 13:27:46.447063: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-20 13:27:46.447099: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-20 13:27:46.447132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-20 13:27:46.447164: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Loading data...
Reading train split.
Reading val split.
Reading test split.


30000it [00:03, 7673.42it/s]


Generating predictions...
938/938 [==============================] - 19s 20ms/step
Rescaling data...
Calculating error...


100%|██████████| 30000/30000 [00:00<00:00, 197319.89it/s]

Systolic BP
73.7% < 15mmHg
61.0% < 10mmHg
38.8% < 5mmHg

Diastolic BP
90.0% < 15mmHg
81.3% < 10mmHg
61.3% < 5mmHg



In [ ]:
import pickle as pkl

with open('test_results.pkl', 'wb') as f:
    pkl.dump(dict(
        error=error,
        ppg=ppg,
        vpg=vpg,
        apg=apg,
        abp=abp,
        pred=pred,
    ), f)

In [ ]:
import pickle as pkl

with open('/home/cam/Documents/database_tools/data/mimic3-data-20230205/data/test_results.pkl', 'rb') as f:
    x = pkl.load(f)

# breezy-wind-80:v8

### MIMIC-III 30k Test Results (max, min method)

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | 72.6%    | 89.5%     |
| < 10mmHg | 59.1%    | 80.2%     |
| <  5mmHg | 36.1%    | 59.4%     |

### VitalDB 30k Test Results

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | %    | %     |
| < 10mmHg | %    | %     |
| <  5mmHg | %    | %     |

# breezy-wind-80:v8

### MIMIC-III 30k Test Results (peak, valley method)

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | 76.9%    | 77.9%     |
| < 10mmHg | 63.5%    | 70.4%     |
| <  5mmHg | 40.5%    | 53.9%     |

In [16]:
import pandas as pd
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

fig = go.Figure()
fig.add_trace(go.Histogram(x=error['sbp_err'], name='SBP Error'))
fig.add_trace(go.Histogram(x=error['dbp_err'], name='DBP Error'))

fig.update_layout(
        barmode='overlay',
        title={
            'text': 'Model Prediction Error Histogram',
            'font': {'size': 20},
        },
        xaxis={'title': 'Error (mmHg)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Number of Samples', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig
fig.update_traces(opacity=0.75)

In [54]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=ppg[20023, :] * 100, name='true'))
fig.add_trace(go.Scatter(y=abp[20023, :], name='pred'))

In [ ]:
from plotly.subplots import make_subplots

i = 18
ppg, vpg, apg, abp, pred = None

fig = make_subplots(rows=2, cols=1, start_cell='top-left')

# Add inputs
fig.add_scatter(
    y=ppg[i, :],
    name='PPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=vpg[i, :] * 10,
    name='VPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=apg[i, :] * 50,
    name='APG',
    row=1,
    col=1,
)

# Add ABP and prediction
fig.add_scatter(
    y=abp[i, :],
    name='ABP (Ground Truth)',
    row=2,
    col=1
)
fig.add_scatter(
    y=pred[i, :],
    name='Prediction',
    row=2,
    col=1,
)

fig.update_layout(
        title={
            'text': 'Model Inputs (PPG, VPG, APG) and Module Output w/ True ABP',
            'font': {'size': 20},
        },
        xaxis2={'title': 'Time (samples)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Amplitude (V)', 'titlefont':{'size': 15}},
        yaxis2={'title': 'Amplitude (mmHg)', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig